# CH7 GPU计算

## 7.6 TensorFlow 的GPU 管理
在TensorFlow中，支持的设备被表示为字符串。
* /cup:0：你机器中的CPU
* /gpu:0：你机器中的GPU（如果有一个的话）
* /gpu:1：你机器中的第二个GPU，等等。

当一个操作被分配给GPU设备时，执行流是有优先级的。

### 程序示例

In [ ]:
import numpy as np
import tensorflow as tf
import datetime

若要在TensorFlow程序中使用GPU，只需在设置操作后输入如下语句：
<br>`with tf.device("/gpu:0"):`

可以写一段程序，查看你的操作和张量被分配到哪一个设备。为实现这一操作，使用下述命令创建一个会话，将log_device_placement参数设置为True：

In [ ]:
log_device_placement = True

In [ ]:
# 然后确定参数n，即需要执行的乘法次数：
n = 10

In [ ]:
# 之后创建一个随机的大型矩阵。A和B的大小分别为10 000×10 000。
# 使用NumPy中的rand函数执行这一操作：
A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')

In [ ]:
# 下面的数组将用于存储运算结果：
c1 = []
c2 = []

In [ ]:
# 此处定义内核矩阵乘法函数，将由GPU执行：
def matpow(M, n):
    if n < 1:
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

之前提到过，必须设置使用哪个GPU，以及用此GPU执行何种操作。

In [ ]:
# 本例中，GPU将计算An+Bn，并将结果保存在c1中：
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

In [ ]:
# 所有元素的和，即An+Bn，储存在c1中。求和操作由CPU执行，因此我们定义如下：
with tf.device('/cpu:0'):
    sum = tf.add_n(c1)

# datetime类统计代码的执行时间：
t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    sess.run(sum, {a:A, b:B})
t2_1 = datetime.datetime.now()

# 运算时间由以下语句显示：
print("GPU computation time: " + str(t2_1-t1_1))

In [ ]:
import numpy as np
import tensorflow as tf
import datetime

log_device_placement = True
n = 10
A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')
c1 = []
c2 = []

def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

with tf.device('/gpu:0'): # For CPU use /cpu:0
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

# If the below code does not work use '/job:localhost/replica:0/task:0/cpu:0' as the GPU device
with tf.device('/cpu:0'):
  sum = tf.add_n(c1) #Addition of all elements in c1, i.e. A^n + B^n

t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                      log_device_placement=log_device_placement)) as sess:
     sess.run(sum, {a:A, b:B})
t2_1 = datetime.datetime.now()

# 运算时间由以下语句显示：
print("GPU computation time: " + str(t2_1-t1_1))

## 7.8 在多GPU 系统上分配单个GPU
如果你的系统里有超过一个GPU，那么TensorFlow会默认选取ID最小的那一块。如果你希望程序在不同的GPU上运行，那么需要进行手动设置，明确指定所用GPU。

例如，可以使用前面讲过的代码更改GPU分配：

In [ ]:
with tf.device('/gpu:1'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

通过这种方式，我们让GPU执行了内核函数。

In [ ]:
# 如果希望在指定设备不存在的情况下，TensorFlow能够自动选择已有的、支持的设备运行操作，
# 那么可以将allow_soft_placement参数设置为True：
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                      log_device_placement=log_device_placement)) as sess:

## 7.9 使用多个GPU
如果你希望在多个GPU上运行TensorFlow，那么可以在构建模型时将特定代码段分配给不同GPU。

例如，如果你有两个GPU，那么可以将前面的代码进行如下分割，将第一个矩阵运算分配给第一个CPU。代码如下：

In [ ]:
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))

第二个矩阵运算被分配给第二个CPU：

In [ ]:
with tf.device('/gpu:1'):
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(b, n))

最后，CPU会管理程序的结果。另外需要注意，我们使用了共享的c1数组来收集结果：

In [ ]:
with tf.device('/cpu:0'):
    sum = tf.add_n(c1)
    print(sum)